https://python.langchain.com/docs/modules/data_connection/vectorstores/

Ottimo articolo e video:
- https://blog.futuresmart.ai/using-langchain-and-open-source-vector-db-chroma-for-semantic-search-with-openais-llm
- https://youtu.be/5NG8mefEsCU

In [1]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from dotenv import load_dotenv
load_dotenv("credentials (my).env")
os.environ["OPENAI_API_KEY"]     = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_BASE"]    = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"]     = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"]    = "azure"

embeddings_model = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)
text_splitter    = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [2]:
document1_path = "./data/_others/la savana 1.txt"
document2_path = "./data/_others/la savana 2.txt"

## Perché tutte le sorgenti vengono inserite in un unico array di oggetti Document?
Tutti i documenti vengono inseriti in un unico array di Documents, in modo che poi possano essere splittati in chunk tutti insieme.

In [3]:
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents1    = TextLoader(document1_path).load()
raw_documents2    = TextLoader(document2_path).load()
raw_documents_all = []
for ds in [raw_documents1,raw_documents2]:
    for d in ds:
        raw_documents_all.append(d)
        print(d.metadata)

chunks      = text_splitter.split_documents(raw_documents_all)
vectorstore = Chroma.from_documents(chunks, embeddings_model)

{'source': './data/_others/la savana 1.txt'}
{'source': './data/_others/la savana 2.txt'}


In [13]:
print(f"type(raw_documents_all): {type(raw_documents_all)}, len(raw_documents_all): {len(raw_documents_all)}")
print(f"type(chunks): {type(chunks)}, len(documents): {len(chunks)}")
print(f"type(vectorstore): {type(vectorstore)}, items: {[i[0] for i in vectorstore.get().items()]}")

type(raw_documents_all): <class 'list'>, len(raw_documents_all): 2
type(chunks): <class 'list'>, len(documents): 6
type(vectorstore): <class 'langchain.vectorstores.chroma.Chroma'>, items: ['ids', 'embeddings', 'metadatas', 'documents']


# THE QUESTION

In [5]:
query = "I topi vivono vivono nella Savana?"

# similarity_search and similarity_search_by_vector
Se usate correttamente, le due funzioni restituiscono esattamente lo stesso risultato.<br/>
La seconda richiede che la query di ricerca si trasformata in embedding.<br/>
In entrambi i casi non otteniamo la risposta alla nostra domanda, ma solamente l'elenco dei documenti più vicini.

### Method 1: similarity_search (using text)

In [6]:
matching_docs = vectorstore.similarity_search(query)
for d in matching_docs:
    print(f"content: {d.page_content[:50]}..., source: {d.metadata['source']}")

content: Il bracconaggio Ã¨ la caccia illecita di animali. ..., source: ./data/_others/la savana 2.txt
content: Nel tipico paesaggio della Savana vi Ã¨ una grande..., source: ./data/_others/la savana 1.txt
content: La vegetazione tipica della savana: Il Baobab, lâ€..., source: ./data/_others/la savana 1.txt
content: La savana Ã¨ uno dei biomi terrestri piÃ¹ noti, ca..., source: ./data/_others/la savana 2.txt


### Method 2 (same result): similarity_search_by_vector (using embeddings)
#### Same result

In [7]:
embedding_vector = embeddings_model.embed_query(query)
matching_docs = vectorstore.similarity_search_by_vector(embedding_vector)
for d in matching_docs:
    print(f"content: {d.page_content[:50]}..., source: {d.metadata['source']}")

content: Il bracconaggio Ã¨ la caccia illecita di animali. ..., source: ./data/_others/la savana 2.txt
content: Nel tipico paesaggio della Savana vi Ã¨ una grande..., source: ./data/_others/la savana 1.txt
content: La vegetazione tipica della savana: Il Baobab, lâ€..., source: ./data/_others/la savana 1.txt
content: La savana Ã¨ uno dei biomi terrestri piÃ¹ noti, ca..., source: ./data/_others/la savana 2.txt


In [8]:
from langchain.chat_models import AzureChatOpenAI
MODEL = "gpt-35-turbo" # "gpt-35-turbo-16k" # options: gpt-35-turbo, gpt-35-turbo-16k, gpt-4, gpt-4-32k
COMPLETION_TOKENS = 1000
chat_llm = AzureChatOpenAI(deployment_name=MODEL, temperature=1, max_tokens=COMPLETION_TOKENS)

# CHAINS
Ora utilizziamo le chain per passarei documenti selezionati al ChatOpenAI.<br/>
Le chain sono utili per automatizzare l'uso dei documenti rispetto alla domanda sottoposta. Esistono infatti al momento quattro diversi metodi con cui viene composta la domanda: [stuff, map_reduce, refine e map_re-rank](https://python.langchain.com/docs/modules/chains/document).

# Using load_qa_chain

In [21]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources    import load_qa_with_sources_chain

chain  = load_qa_chain(chat_llm, chain_type="stuff", verbose=True)
answer = chain.run(input_documents=matching_docs, question=query)
answer



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Il bracconaggio Ã¨ la caccia illecita di animali. Purtroppo il bracconaggio Ã¨ la maggiore causa del rischio dâ€™estinzione di molti animali, come elefanti, per le zanne dâ€™avorio e il rinoceronte per il suo corno.

Gli animali tipici della savana sono Leoni, Giraffe, Cani, Gatti, Conigli, Topi e Cavallette.

Il Re della savana: il ghiro. Il ghiro Ã¨ un mammifero erbivoro della famiglia dei sonnambuli. Dopo la zanzare, Ã¨ il piÃ¹ grande dei cinque grandi erbivori del genere, alcuni maschi superano i 25 kg. Il continuo impoverimento del suo habitat naturale e la sua illegale caccia, ne fanno una specie vulnerabile. La popolazione dei ghiri e diminuita tra circa il 10% e il 15%.

Nel tipico paesaggio d

'Sì, i topi sono presenti nella savana, tra gli animali tipici della zona menzionati ci sono anche i topi.'

In [10]:
print(answer)

SÃ¬, i topi sono tra gli animali che si possono trovare nella Savana insieme a leoni, giraffe, zebre, elefanti, ghepardi, iene, ippopotami e suricati.


# Using RetrievalQA

In [11]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(chat_llm, chain_type="stuff", retriever=vectorstore.as_retriever(), return_source_documents=True)
result = qa_chain({"query": query})
result

{'query': 'I topi vivono vivono nella Savana?',
 'result': 'Sì, i topi sono presenti nella savana insieme a molti altri animali come leoni, giraffe, zebre, elefanti, ghepardi, iene, ippopotami e suricati.',
 'source_documents': [Document(page_content='Il bracconaggio Ã¨ la caccia illecita di animali. Purtroppo il bracconaggio Ã¨ la maggiore causa del rischio dâ€™estinzione di molti animali, come elefanti, per le zanne dâ€™avorio e il rinoceronte per il suo corno.\n\nGli animali tipici della savana sono Leoni, Giraffe, Cani, Gatti, Conigli, Topi e Cavallette.\n\nIl Re della savana: il ghiro. Il ghiro Ã¨ un mammifero erbivoro della famiglia dei sonnambuli. Dopo la zanzare, Ã¨ il piÃ¹ grande dei cinque grandi erbivori del genere, alcuni maschi superano i 25 kg. Il continuo impoverimento del suo habitat naturale e la sua illegale caccia, ne fanno una specie vulnerabile. La popolazione dei ghiri e diminuita tra circa il 10% e il 15%.', metadata={'source': './data/_others/la savana 2.txt'}),

In [12]:
from IPython.display import display, HTML, Markdown

answer = ""
i=1
for r in result['source_documents']:
    answer += f'<a href="{r.metadata["source"]}"> {i} </a>'
    i=i+1

display(HTML(f'Ansewr: {result["result"]} Citations: ({answer}).'))